In [ ]:
import numpy as np
import pandas as pd

def calculate_differences():
   surface_data = {
       '100': (0.462, 0.463),
       '110': (0.501, 0.495), 
       '111': (0.544, 0.532),
       '210': (0.506, 0.534),
       '211': (0.538, 0.534),
       '221': (0.518, 0.526),
       '310': (0.497, 0.496),
       '311': (0.527, 0.521),
       '320': (0.504, 0.498),
       '321': (0.534, 0.532),
       '322': (0.535, 0.535),
       '331': (0.521, 0.518),
       '332': (0.524, 0.520)
   }

   results = []
   for surface in sorted(surface_data.keys()):
       mine, mp = surface_data[surface]
       percent_diff = abs(mine - mp) / mp * 100
       results.append({
           'Surface': surface,
           'My Value': mine,
           'MP Value': mp,
           'Difference (%)': percent_diff
       })
   
   df = pd.DataFrame(results)
   pd.set_option('display.float_format', '{:.3f}'.format)
   print(df.to_string(index=False))
   print(f"\nAverage difference: {df['Difference (%)'].mean():.3f}%")
   print(f"Maximum difference: {df['Difference (%)'].max():.3f}%") 
   print(f"Minimum difference: {df['Difference (%)'].min():.3f}%")
   print(f"Standard deviation: {df['Difference (%)'].std():.3f}%")

if __name__ == "__main__":
   calculate_differences()

In [ ]:
import os
import re
from typing import Dict, Tuple, Optional, Union
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb, Normalize
from matplotlib.colorbar import ColorbarBase
from pymatgen.core import Structure
from wulffpack import SingleCrystal


def extract_surface_energies_pbs(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing PBS output files.
    
    Args:
        parent_dir: Directory path containing subdirectories with PBS output files.
        
    Returns:
        Dictionary mapping Miller indices tuples to surface energy values in J/m^2.
    """
def extract_surface_energies_slurm(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing SLURM output files.
    
    Args:
        parent_dir: Path to the parent directory containing Miller index subdirectories
        
    Returns:
        Dictionary mapping Miller indices tuples to their corresponding surface energies in J/m^2
        
    Example:
        >>> energies = extract_surface_energies_slurm("/path/to/calculations")
        >>> print(energies[(1, 1, 1)])
        2.34
    """
    results = {}
    
    for dirname in (d for d in os.listdir(parent_dir) if d.startswith('[')):
        full_path = os.path.join(parent_dir, dirname)
        if not os.path.isdir(full_path):
            continue
            
        miller_match = re.search(r'\[(\d{3})\]', dirname)
        if not miller_match:
            continue
            
        miller_indices = miller_match.group(1)
        miller_tuple = tuple(int(miller_indices[i]) for i in range(3))
        
        for file in os.listdir(full_path):
            if file.startswith('slurm-') and file.endswith('.out'):
                slurm_path = os.path.join(full_path, file)
                try:
                    with open(slurm_path, 'r') as f:
                        content = f.read()
                        energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                        if energy_match:
                            results[miller_tuple] = float(energy_match.group(1))
                            break
                except Exception as e:
                    print(f"Error reading {slurm_path}: {e}")
    
    return results


def extract_surface_energies(parent_dir: str, system: str = 'pbs') -> Dict[Tuple[int, int, int], float]:
    """Extract surface energies from a directory structure containing job output files.
    
    Args:
        parent_dir: Directory path containing subdirectories with output files.
        system: Job system type ('pbs' or 'slurm'). Defaults to 'pbs'.
        
    Returns:
        Dictionary mapping Miller indices tuples to surface energy values in J/m^2.
        
    Raises:
        ValueError: If system type is not recognized.
    """
    if system.lower() == 'pbs':
        return extract_surface_energies_pbs(parent_dir)
    elif system.lower() == 'slurm':
        return extract_surface_energies_slurm(parent_dir)
    else:
        raise ValueError(f"Unrecognized system type: {system}. Use 'pbs' or 'slurm'.")


def extract_surface_energies_pbs(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
    results = {}
    
    for dirname in os.listdir(parent_dir):
        if not dirname.startswith('[') or not dirname.endswith(']'):
            continue
            
        full_path = os.path.join(parent_dir, dirname)
        if not os.path.isdir(full_path):
            continue
        
        try:
            miller_str = dirname.strip('[]')
            if len(miller_str) != 3:
                continue
            miller_tuple = tuple(int(miller_str[i]) for i in range(3))
            
            output_file = f"miller-{miller_str}.o"
            matching_files = [f for f in os.listdir(full_path) if f.startswith(output_file)]
            
            if matching_files:
                pbs_path = os.path.join(full_path, matching_files[0])
                try:
                    with open(pbs_path, 'r') as f:
                        content = f.read()
                        energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                        if energy_match:
                            results[miller_tuple] = float(energy_match.group(1))
                except Exception as e:
                    print(f"Error reading {pbs_path}: {e}")
                        
        except Exception as e:
            print(f"Error processing directory {dirname}: {e}")
            continue
    
    return results


def simple_wulff(
    surface_energies: Dict[Tuple[int, int, int], float],
    bulk_path: str,
    output_path: Optional[str] = None,
    view_angles: Tuple[float, float, float] = (45, 45, 0)
) -> SingleCrystal:
    bulk_structure = Structure.from_file(bulk_path)
    bulk_structure_ase = bulk_structure.to_ase_atoms()
    particle = SingleCrystal(surface_energies, primitive_structure=bulk_structure_ase)
    
    # Analyze shape
    facet_fractions = particle.facet_fractions
    active_surfaces = {miller: surface_energies[miller] 
                      for miller in facet_fractions.keys() if facet_fractions[miller] > 0}
    
    # Print shape analysis
    print("\nShape Analysis:")
    print(f"Number of unique faces: {len(active_surfaces)}")
    print("\nActive Miller indices:")
    for miller in active_surfaces.keys():
        print(f"({miller[0]}{miller[1]}{miller[2]})")
        
    # Analyze symmetry
    unique_sums = {sum(miller) for miller in active_surfaces.keys()}
    print(f"\nUnique sums of Miller indices: {sorted(unique_sums)}")
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    color_order = sorted(active_surfaces.keys(), 
                        key=lambda x: (sum(x), x[0], x[1], x[2]))
    
    n_surfaces = len(color_order)
    colors = {}
    
    for i, miller in enumerate(color_order):
        colors[miller] = plt.cm.tab20(i % 20)
    
    particle.make_plot(
        ax,
        colors=colors,
        alpha=1.0,
        linewidth=2.0
    )
    
    ax.view_init(*view_angles)
    ax.set_axis_off()
    
    sorted_by_percentage = sorted(facet_fractions.items(),
                                key=lambda x: x[1],
                                reverse=True)
    
    legend_elements = []
    legend_labels = []
    
    for miller, percentage in sorted_by_percentage:
        if miller in colors:
            patch = plt.Rectangle((0,0), 1, 1, facecolor=colors[miller], edgecolor='black')
            legend_elements.append(patch)
            legend_labels.append(f"({miller[0]}{miller[1]}{miller[2]}) - {percentage*100:.1f}%")
    
    ax.legend(
        legend_elements,
        legend_labels,
        title="Miller Indices",
        loc='center left',
        bbox_to_anchor=(1.1, 0.5),
        fontsize=12,
        title_fontsize=14
    )
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    return particle


def main() -> None:
    """Main execution function for processing surface energies and generating Wulff construction visualisation."""
    SURFACES_DIR = "/home/ba3g18/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/[-1.75V]"
    BULK_PATH = "/home/ba3g18/Documents/Git-Repos/lithium-nanoparticles/analysis/Li.cif"
    OUTPUT_PATH = "/home/ba3g18/Documents/Git-Repos/lithium-nanoparticles/analysis/Wulff_shapes/neg_1.75V_wulff.png"
    SYSTEM_TYPE = "slurm" # or "slurm"
    
    surface_energies = extract_surface_energies(SURFACES_DIR, system=SYSTEM_TYPE)
    print(surface_energies)
    if surface_energies:
        particle = simple_wulff(
            surface_energies=surface_energies,
            bulk_path=BULK_PATH,
            output_path=OUTPUT_PATH
        )
    else:
        print("No surface energies found in the directories")


if __name__ == "__main__":
    main()

In [ ]:
import os
from typing import Dict, Tuple, Optional
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Structure
from wulffpack import SingleCrystal

def simple_wulff(
    surface_energies: Dict[Tuple[int, int, int], float],
    bulk_path: str,
    output_path: Optional[str] = None,
    view_angles: Tuple[float, float, float] = (45, 45, 0)
) -> SingleCrystal:
    bulk_structure = Structure.from_file(bulk_path)
    bulk_structure_ase = bulk_structure.to_ase_atoms()
    particle = SingleCrystal(surface_energies, primitive_structure=bulk_structure_ase)
    
    facet_fractions = particle.facet_fractions
    active_surfaces = {miller: surface_energies[miller] 
                      for miller in facet_fractions.keys() if facet_fractions[miller] > 0}
    
    # Print shape analysis
    print("\nShape Analysis:")
    print(f"Number of unique faces: {len(active_surfaces)}")
    print("\nActive faces and their surface areas:")
    for miller, fraction in sorted(facet_fractions.items(), key=lambda x: x[1], reverse=True):
        if fraction > 0:
            print(f"({miller[0]},{miller[1]},{miller[2]}): {fraction*100:.2f}%")
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    color_order = sorted(active_surfaces.keys(), 
                        key=lambda x: (sum(x), x[0], x[1], x[2]))
    
    colors = {}
    for i, miller in enumerate(color_order):
        colors[miller] = plt.cm.tab20(i % 20)
    
    particle.make_plot(
        ax,
        colors=colors,
        alpha=1.0,
        linewidth=2.0
    )
    
    ax.view_init(*view_angles)
    ax.set_axis_off()
    
    legend_elements = []
    legend_labels = []
    
    for miller, percentage in sorted(facet_fractions.items(), key=lambda x: x[1], reverse=True):
        if miller in colors and percentage > 0:
            patch = plt.Rectangle((0,0), 1, 1, facecolor=colors[miller], edgecolor='black')
            legend_elements.append(patch)
            legend_labels.append(f"({miller[0]}{miller[1]}{miller[2]}) - {percentage*100:.1f}%")
    
    ax.legend(
        legend_elements,
        legend_labels,
        title="Miller Indices",
        loc='center left',
        bbox_to_anchor=(1.1, 0.5),
        fontsize=12,
        title_fontsize=14
    )
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    return particle

def main():
    # Define surface energies
    surface_energies = {
        (1, 0, 0): 0.0529,
        (1, 1, 0): 0.0334,
        (1, 1, 1): 0.103,
        (2, 1, 0): 0.0618,
        (2, 1, 1): 0.0605,
        (2, 2, 1): 0.0827,
        (3, 1, 0): 0.0647,
        (3, 1, 1): 0.09145,
        (3, 2, 0): 0.0569,
        (3, 2, 1): 0.0665,
        (3, 2, 2): 0.0924,
        (3, 3, 1): 0.0744,
        (3, 3, 2): 0.0817
    }
    
    BULK_PATH = "/home/ba3g18/Documents/Git-Repos/lithium-nanoparticles/analysis/Li.cif"  # Make sure this file exists in your directory
    OUTPUT_PATH = "wulff_shape.png"  # Optional: Set to None to display instead of save
    
    particle = simple_wulff(
        surface_energies=surface_energies,
        bulk_path=BULK_PATH,
        output_path=OUTPUT_PATH
    )
    
    return particle

if __name__ == "__main__":
    main()

In [ ]:
import os
import re
from typing import Dict, Tuple, Optional, Union
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb, Normalize
from matplotlib.colorbar import ColorbarBase
from pymatgen.core import Structure
from wulffpack import SingleCrystal



def simple_wulff(
    surface_energies: Dict[Tuple[int, int, int], float],
    bulk_path: str,
    output_path: Optional[str] = None,
    view_angles: Tuple[float, float, float] = (45, 45, 0),
    colorbar_x: float = 0.38,
    colorbar_y: float = 0.15,
    colorbar_width: float = 0.45
) -> SingleCrystal:
    """Create a Wulff shape visualization with surface energy-based coloring and colorbar.
    
    Args:
        surface_energies: Dictionary mapping Miller indices to surface energies.
        bulk_path: Path to bulk structure file.
        output_path: Optional path to save the visualization.
        view_angles: Tuple of angles (elevation, azimuth, roll) for viewing the shape.
        colorbar_x: X-position of the colorbar.
        colorbar_y: Y-position of the colorbar.
        colorbar_width: Width of the colorbar.
        
    Returns:
        SingleCrystal object representing the Wulff construction.
    """
    bulk_structure = Structure.from_file(bulk_path)
    bulk_structure_ase = bulk_structure.to_ase_atoms()
    particle = SingleCrystal(surface_energies, primitive_structure=bulk_structure_ase, tol=1e-10, symprec=1e-10)
    
    facet_fractions = particle.facet_fractions
    active_surfaces = {miller: surface_energies[miller] 
                      for miller in facet_fractions.keys() if facet_fractions[miller] > 0}
    
    min_energy = min(active_surfaces.values())
    max_energy = max(active_surfaces.values())
    
    fig = plt.figure(figsize=(12, 10))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    ax = fig.add_subplot(gs[0], projection='3d')
    
    color_order = sorted(active_surfaces.keys(), 
                        key=lambda x: (sum(x), x[0], x[1], x[2]))
    
    n_surfaces = len(color_order)
    colors = {}
    saturation_factor = 0.85
    
    for i, miller in enumerate(color_order):
        color_val = i / (n_surfaces - 1) if n_surfaces > 1 else 0.5
        rgb_color = plt.cm.viridis(color_val)[:3]
        hsv_color = rgb_to_hsv(rgb_color)
        hsv_color[1] *= saturation_factor
        rgb_color = hsv_to_rgb(hsv_color)
        colors[miller] = (*rgb_color, 1.0)
    
    particle.make_plot(ax, colors=colors, alpha=0.95)
    ax.view_init(*view_angles)
    ax.set_axis_off()
    
    legend_ax = fig.add_subplot(gs[1])
    legend_ax.axis('off')
    
    sorted_by_percentage = sorted(facet_fractions.items(),
                                key=lambda x: x[1],
                                reverse=True)
    
    legend_elements = [plt.Rectangle((0, 0), 1, 1, facecolor=colors[m], alpha=0.9)
                      for m, _ in sorted_by_percentage if m in colors]
    
    legend_labels = [f"({m[0]}{m[1]}{m[2]}) - {p*100:.1f}%" 
                    for m, p in sorted_by_percentage if m in colors]
    
    legend = legend_ax.legend(
        legend_elements,
        legend_labels,
        title="Miller Indices",
        loc='center left',
        bbox_to_anchor=(-0.2, 0.45),
        bbox_transform=legend_ax.transAxes,
        fontsize=15,
        title_fontsize=17,
        labelspacing=1.32,
        handletextpad=1.1,
        handlelength=1.65,
        frameon=True,
        edgecolor='lightgray'
    )
    
    cbar_ax = fig.add_axes([
        colorbar_x - (colorbar_width/2),
        colorbar_y,
        colorbar_width,
        0.04
    ])
    
    norm = Normalize(vmin=min_energy, vmax=max_energy)
    cbar = ColorbarBase(cbar_ax, cmap=plt.cm.viridis, norm=norm, orientation='horizontal')
    cbar.ax.tick_params(labelsize=12)
    
    fig.text(
        colorbar_x,
        colorbar_y - 0.06,
        "Surface Energy (J/m²)",
        fontsize=14,
        ha='center'
    )
    
    plt.subplots_adjust(left=0.1, right=0.9, top=0.95, bottom=0.15)
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()
    
    return particle


def main() -> None:
    """Main execution function for processing surface energies and generating Wulff construction visualisation."""
    BULK_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Li.cif"
    OUTPUT_PATH = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/analysis/Wulff_shapes/vacuum_wulff.png"
    
    surface_energies = {(1, 0, 0): 0.4616, (3, 1, 0): 0.4935, (1, 1, 0): 0.4945, (3, 2, 0): 0.4974, (2, 1, 0): 0.5017, (3, 3, 1): 0.5176, (3, 3, 2): 0.5200, (2, 2, 1): 0.5210, (3, 1, 1): 0.5223, (3, 2, 1): 0.5286, (3, 2, 2): 0.5351, (2, 1, 1): 0.5376, (1, 1, 1): 0.5404, }
    print(surface_energies)
    if surface_energies:
        particle = simple_wulff(
            surface_energies=surface_energies,
            bulk_path=BULK_PATH,
            output_path=OUTPUT_PATH
        )
    else:
        print("No surface energies found in the directories")


if __name__ == "__main__":
    main()

In [ ]:
import os
import re
import pandas as pd
from ase.io import read
from typing import Dict, List, Optional, Union

def get_pbs_energy(path: str, miller_str: str) -> Optional[float]:
    """Extract surface energy from PBS output files.
    
    Args:
        path: Directory path containing PBS output files
        miller_str: Miller index string without brackets
        
    Returns:
        Surface energy value in J/m^2 if found, None otherwise
    """
    output_file = f"miller-{miller_str}.o"
    matching_files = [f for f in os.listdir(path) if f.startswith(output_file)]
    
    if matching_files:
        pbs_path = os.path.join(path, matching_files[0])
        try:
            with open(pbs_path, 'r') as f:
                content = f.read()
                energy_match = re.search(r'Surface energy:\s*([-]?\d+\.\d+)\s*J/m\^2', content)
                if energy_match:
                    return float(energy_match.group(1))
        except Exception as e:
            print(f"Error reading {pbs_path}: {e}")
    return None

def get_slurm_energy(path: str) -> Optional[float]:
    """Extract surface energy from SLURM output files.
    
    Args:
        path: Directory path containing SLURM output files
        
    Returns:
        Surface energy value in J/m^2 if found, None otherwise
    """
    for file in os.listdir(path):
        if file.startswith('slurm-') and file.endswith('.out'):
            slurm_path = os.path.join(path, file)
            try:
                with open(slurm_path, 'r') as f:
                    content = f.read()
                    energy_match = re.search(r'Surface energy:\s*([-]?\d+\.\d+)\s*J/m\^2', content)
                    if energy_match:
                        return float(energy_match.group(1))
            except Exception as e:
                print(f"Error reading {slurm_path}: {e}")
    return None

def create_surface_energy_table(base_dir: str) -> pd.DataFrame:
    """Generate a DataFrame containing surface energies and PZC values for different facets.
    
    Args:
        base_dir: Base directory containing voltage subdirectories
        
    Returns:
        DataFrame with facet indices, PZC values, and surface energies at different voltages
    """
    data = []
    voltage_values = [-2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0]
    neutral_dir = os.path.join(base_dir, 'Neutral')
    miller_indices = sorted([d for d in os.listdir(neutral_dir) if d.startswith('[')])
    
    for miller in miller_indices:
        row = {'Facet': miller}
        miller_str = miller.strip('[]')
        
        try:
            pwo_path = os.path.join(neutral_dir, miller, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            row['PZC'] = atoms.calc.get_fermi_level()
        except Exception as e:
            row['PZC'] = None
            
        for v in voltage_values:
            col = f"{v}_VLi"
            row[col] = None
            dir_name = 'Neutral' if v == 0.0 else f"[{v}V]"
            voltage_dir = os.path.join(base_dir, dir_name)
            
            if os.path.exists(voltage_dir):
                miller_path = os.path.join(voltage_dir, miller)
                if os.path.exists(miller_path):
                    energy = get_pbs_energy(miller_path, miller_str) if v in [0.5, 1.0] else get_slurm_energy(miller_path)
                    row[col] = energy
                    
        data.append(row)
    
    df = pd.DataFrame(data)
    cols = ['Facet', 'PZC'] + [f'{v}_VLi' for v in voltage_values]
    return df[cols]

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    df = create_surface_energy_table(base_dir)
    print("\nLi Surface Energies and PZC Values:")
    print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
    df.to_csv("li_surface_energies.csv", index=False)

In [ ]:
import os
import re
import pandas as pd
from typing import Dict, Tuple

def extract_surface_energies_slurm(parent_dir: str) -> Dict[Tuple[int, int, int], float]:
   results = {}
   for dirname in (d for d in os.listdir(parent_dir) if d.startswith('[')):
       full_path = os.path.join(parent_dir, dirname)
       if not os.path.isdir(full_path):
           continue
       miller_match = re.search(r'\[(\d{3})\]', dirname)
       if not miller_match:
           continue
       miller_indices = miller_match.group(1)
       miller_tuple = tuple(int(miller_indices[i]) for i in range(3))
       
       for file in os.listdir(full_path):
           if file.startswith('slurm-') and file.endswith('.out'):
               slurm_path = os.path.join(full_path, file)
               try:
                   with open(slurm_path, 'r') as f:
                       content = f.read()
                       energy_match = re.search(r'Surface energy:\s*(\d+\.\d+)\s*J/m\^2', content)
                       if energy_match:
                           results[miller_tuple] = round(float(energy_match.group(1)), 4)
                           break
               except Exception as e:
                   print(f"Error reading {slurm_path}: {e}")
   return results

def create_miller_tuple(miller_str: str) -> Tuple[int, int, int]:
   miller_int = int(float(miller_str))
   return (miller_int//100, (miller_int//10)%10, miller_int%10)

def create_comparison_table(neutral_dir: str, vacuum_file: str) -> pd.DataFrame:
   solvated_energies = extract_surface_energies_slurm(neutral_dir)
   df_vacuum = pd.read_csv(vacuum_file, sep='\t', skiprows=0)
   
   data = []
   delta_gammas = []
   for _, row in df_vacuum.iterrows():
       miller_str = str(row['Slab'])
       miller_tuple = create_miller_tuple(miller_str)
       
       if miller_tuple in solvated_energies:
           solv_energy = solvated_energies[miller_tuple]
           vac_energy = row['Surface Energy (J/m^2)']
           delta_gamma = round(solv_energy - vac_energy, 4)
           delta_gammas.append(abs(delta_gamma))
           
           data.append({
               'Miller Index': f"[{miller_str.split('.')[0]}]",
               'Vacuum γ (J/m²)': vac_energy,
               'Solvated γ (J/m²)': solv_energy,
               'Δγ (J/m²)': delta_gamma
           })
   
   df = pd.DataFrame(data).sort_values('Miller Index')
   abs_range = round(max(delta_gammas) - min(delta_gammas), 4)
   print(f"\nAbsolute Δγ range: {abs_range} J/m²")
   print(f"Maximum absolute Δγ: {max(delta_gammas)} J/m²")
   print(f"Minimum absolute Δγ: {min(delta_gammas)} J/m²")
   return df

neutral_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/Neutral"
vacuum_file = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/Vacuum/surface_energies.txt"

comparison_table = create_comparison_table(neutral_dir, vacuum_file)
print("\nSurface Energy Comparison Table:")
print(comparison_table.to_string(index=False))
comparison_table.to_csv("surface_energy_comparison.csv", index=False)

In [ ]:
import os
from ase.io import read
import pandas as pd
from typing import Dict, Tuple

def get_fermi_levels(directory: str) -> Dict[str, float]:
    """Extract Fermi levels from neutral and vacuum calculations."""
    results = {}
    
    # Process Neutral directory
    neutral_dir = os.path.join(directory, 'Neutral')
    for dirname in (d for d in os.listdir(neutral_dir) if d.startswith('[')):
        try:
            pwo_path = os.path.join(neutral_dir, dirname, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            results[f"neutral_{dirname}"] = atoms.calc.get_fermi_level()
        except Exception as e:
            print(f"Error reading neutral {dirname}: {e}")
    
    # Process Vacuum directory
    vacuum_dir = os.path.join(directory, 'Vacuum')
    for dirname in os.listdir(vacuum_dir):
        if dirname.startswith('Surface_'):
            try:
                miller = dirname.split('_')[1]
                pwo_path = os.path.join(vacuum_dir, dirname, 'espresso.pwo')
                atoms = read(pwo_path)
                results[f"vacuum_{miller}"] = atoms.calc.get_fermi_level()
            except Exception as e:
                print(f"Error reading vacuum {dirname}: {e}")
    
    return results

def create_fermi_comparison(base_dir: str) -> pd.DataFrame:
    fermi_levels = get_fermi_levels(base_dir)
    data = []
    delta_fermis = []
    
    for miller in sorted(set(key.split('_')[1] for key in fermi_levels.keys() if key.startswith('neutral_'))):
        neutral_key = f"neutral_{miller}"
        vacuum_key = f"vacuum_{miller.strip('[]')}"
        
        if neutral_key in fermi_levels and vacuum_key in fermi_levels:
            neutral_fermi = fermi_levels[neutral_key]
            vacuum_fermi = fermi_levels[vacuum_key]
            delta = round(neutral_fermi - vacuum_fermi, 4)
            delta_fermis.append(abs(delta))
            
            data.append({
                'Miller Index': miller,
                'Vacuum E_f (eV)': vacuum_fermi,
                'Solvated E_f (eV)': neutral_fermi,
                'ΔE_f (eV)': delta
            })
    
    df = pd.DataFrame(data).sort_values('Miller Index')
    
    abs_range = round(max(delta_fermis) - min(delta_fermis), 4)
    print(f"\nAbsolute ΔE_f range: {abs_range} eV")
    print(f"Maximum absolute ΔE_f: {max(delta_fermis)} eV")
    print(f"Minimum absolute ΔE_f: {min(delta_fermis)} eV")
    
    return df

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    comparison_table = create_fermi_comparison(base_dir)
    print("\nFermi Level Comparison Table:")
    print(comparison_table.to_string(index=False))
    comparison_table.to_csv("fermi_level_comparison.csv", index=False)

In [ ]:
import os
from ase.io import read
import numpy as np

def calculate_pzc_differences(base_dir: str, reference_value: float = -3.404):
    """Calculate PZC differences (reference - fermi) for each facet."""
    neutral_dir = os.path.join(base_dir, 'Neutral')
    miller_indices = sorted([d for d in os.listdir(neutral_dir) if d.startswith('[')])
    
    pzc_diffs = {}
    for miller in miller_indices:
        try:
            pwo_path = os.path.join(neutral_dir, miller, 'Surface', 'espresso.pwo')
            atoms = read(pwo_path)
            fermi_level = atoms.calc.get_fermi_level()
            pzc_diffs[miller] = reference_value - fermi_level
        except Exception as e:
            print(f"Error processing {miller}: {e}")
    
    # Find smallest and largest differences
    min_diff_idx = min(pzc_diffs.items(), key=lambda x: abs(x[1]))
    max_diff_idx = max(pzc_diffs.items(), key=lambda x: abs(x[1]))
    
    print("\nPZC Differences (Reference - Fermi):")
    for miller, diff in pzc_diffs.items():
        print(f"{miller}: {diff:.4f} V_{{Li/Li+}}")
    
    print(f"\nThe PZC differences from Li/Li+ reference range from {min(pzc_diffs.values()):.4f} to {max(pzc_diffs.values()):.4f} V_{{Li/Li+}}")
    print(f"Smallest difference: {min_diff_idx[0]} at {min_diff_idx[1]:.4f} V_{{Li/Li+}}")
    print(f"Largest difference: {max_diff_idx[0]} at {max_diff_idx[1]:.4f} V_{{Li/Li+}}")
    
    return pzc_diffs

if __name__ == "__main__":
    base_dir = "/Users/bdayers/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production"
    pzc_diffs = calculate_pzc_differences(base_dir)

In [ ]:
E_vac = 0.93 
E_fermi = -2.8050
shift = 4.44
work_function = E_fermi - E_vac
NHE = work_function -  shift
print("NHE", NHE)

Li_ref = -3.404 
Li_vs_NHE = NHE - Li_ref

print("Li_vs_NHE", Li_vs_NHE)
experimental = -3.04
delta = experimental - Li_vs_NHE
print("delta", delta)


In [ ]:
from ase.io import read, write
from ase.visualize import view
from acat.adsorption_sites import SlabAdsorptionSites
from acat.build.adlayer import min_dist_coverage_pattern as mindcp
from ase.geometry import get_layers
from acat.settings import CustomSurface
import numpy as np

def analyse_and_fluorinate_surface(filepath):
    """
    Analyses a Li surface structure and adds F atoms to all available adsorption sites.
    Returns the fluorinated structure and prints analysis of sites and F placement.
    """
    slab = read(filepath)
    slab.pbc = [True, True, True]
    slab.center() 

    layers, _ = get_layers(slab, (0, 0, 1), tolerance=0.1)
    n_layers = len(np.unique(layers))
    print(f"\nDetected {n_layers} layers in the structure")

    custom_surface = CustomSurface(slab, n_layers=n_layers)
    sas = SlabAdsorptionSites(slab, 
                             surface=custom_surface,
                             surrogate_metal='Li',
                             both_sides=True)

    all_sites = sas.get_sites()
    site_types = {}
    for site in all_sites:
        site_type = site['site']
        if site_type not in site_types:
            site_types[site_type] = []
        site_types[site_type].append(site)

    available_site_types = list(site_types.keys())

    print("\nPossible adsorption sites:")
    print("-" * 50)
    for site_type, sites in site_types.items():
        print(f"\n{site_type.upper()} sites:")
        print(f"  Number of sites: {len(sites)}")
        print(f"  Example site details:")
        print(f"    Position: {sites[0]['position']}")
        print(f"    Contributing Li atoms: {sites[0]['indices']}")

    atoms = mindcp(slab, 
                  adsorbate_species='F',
                  surface=custom_surface,
                  min_adsorbate_distance=2.0,
                  both_sides=True,
                  site_types=available_site_types)

    li_mask = [atom.symbol == 'Li' for atom in atoms]
    f_mask = [atom.symbol == 'F' for atom in atoms]
    
    f_positions = atoms.positions[f_mask]
    li_z_coords = atoms.positions[li_mask, 2]
    
    z_mid = (np.max(li_z_coords) + np.min(li_z_coords)) / 2
    f_top = f_positions[f_positions[:,2] > z_mid]
    f_bottom = f_positions[f_positions[:,2] < z_mid]

    print("\nSymmetry Analysis:")
    print(f"F atoms above midpoint: {len(f_top)}")
    print(f"F atoms below midpoint: {len(f_bottom)}")
    print(f"Symmetry deviation: {abs(len(f_top) - len(f_bottom))} atoms")
    
    top_heights = np.unique(np.round(f_top[:,2] - z_mid, 2))
    bottom_heights = np.unique(np.round(z_mid - f_bottom[:,2], 2))
    
    print("\nF Height Analysis:")
    print(f"Top surface F heights: {sorted(top_heights)}")
    print(f"Bottom surface F heights: {sorted(bottom_heights)}")
    print(f"Height symmetry deviation: {np.abs(np.mean(top_heights) - np.mean(bottom_heights)):.3f} Å")
    
    return atoms

if __name__ == "__main__":
    structure = analyse_and_fluorinate_surface(
        "/home/ba3g18/Documents/Git-Repos/lithium-nanoparticles/data/bondi-production/[-1.75V]/[100]/Surface/espresso.pwo"
    )
    view(structure)